In [1]:
import random
import pennylane as qml
from pennylane import numpy as np
from myutils import *

#Magic Command, so changes in myutils module are reloaded
%load_ext autoreload
%autoreload 1
%aimport myutils

# Loading the data

In [27]:
alldata=  Datasets.get_preprocessed_datasets("PCA")["PCA"]



In [28]:
subset_selection = ["1,6","3,8","4,6","0,1","3,6","6,9"]

- 1,6 -> easyiest
- 3,8 -> schwerste
- 4,9 -> schwere drittel
- 0,1 -> leichte drittel
- 3,6 -> mitte
- 6,9 -> mitte unserzahlenpaar

In [30]:
n = 100
n_test = 500


for subset in subset_selection:
    #scale data 0, 2*np
    data = alldata[subset]
    data["x_train"] = Preprocessing.minmax_scaler(data["x_train"] , min = 0,max = 2 * np.pi)
    data["x_test"] = Preprocessing.minmax_scaler(data["x_test"], min = 0,max = 2 * np.pi)

    for type in ["y_train_binary","y_test_binary"]:
        quantum_convert = []
        for i in range(len(data[type])):
            if data[type][i] == 0:
                quantum_convert.append(np.array([0,1]))
            else:
                quantum_convert.append(np.array([1,0]))

        data[type+"_quantum"] = np.array(quantum_convert)   

    for round in range(3):
          # get random sample and split data
        training_indices = np.array(random.sample(range(len(data["x_train"])), int(n))).numpy()
        test_indices = np.array(random.sample(range(len(data["x_test"])), int(n_test))).numpy()
        

        Helpers.store(training_indices,"data/indices/{}/{}" .format(subset,round), "training_indices")
        Helpers.store(test_indices,"data/indices/{}/{}" .format(subset,round), "test_indices")
    


In [32]:
for subset in ["1,6","3,8","0,1","3,6","6,9"]:

    #scale data 0, 2*np
    data = alldata[subset]
    data["x_train"] = Preprocessing.minmax_scaler(data["x_train"] , min = 0,max = 2 * np.pi)
    data["x_test"] = Preprocessing.minmax_scaler(data["x_test"], min = 0,max = 2 * np.pi)

    for type in ["y_train_binary","y_test_binary"]:
        quantum_convert = []
        for i in range(len(data[type])):
            if data[type][i] == 0:
                quantum_convert.append(np.array([0,1]))
            else:
                quantum_convert.append(np.array([1,0]))

        data[type+"_quantum"] = np.array(quantum_convert)   

    for round in range(3):


        print("subset: ",subset,"round: ",round)

        # get random sample and split data
        training_indices = np.load("data/indices/{}/{}/training_indices.npy" .format(subset,round))
        test_indices = np.load("data/indices/{}/{}/test_indices.npy" .format(subset,round))

        data["x_train_subset"] = data["x_train"][training_indices]
        data["y_train_binary_quantum_subset"] = data["y_train_binary_quantum"][training_indices]

        data["x_test_subset"] = data["x_test"][test_indices]
        data["y_test_binary_quantum_subset"] = data["y_test_binary_quantum"][test_indices]
    
        # create and train circuit
        qcircuit = QuatumCircuit()
        hist,params = qcircuit.train(x_train=data["x_train_subset"],
        y_train=data["y_train_binary_quantum_subset"],
        x_test=data["x_test"],
        y_test=data["y_test_binary_quantum"],
        epochs=10,
        verbose=0)

        Helpers.store(hist,"measure/QCircuit/PCA/{}/{}".format(subset,round),"hist")
        Helpers.store(params.numpy(),"measure/QCircuit/PCA/{}/{}".format(subset,round),"params")

subset:  1,6 round:  0
subset:  1,6 round:  1
subset:  1,6 round:  2
subset:  3,8 round:  0
subset:  3,8 round:  1
subset:  3,8 round:  2
subset:  0,1 round:  0
subset:  0,1 round:  1
subset:  0,1 round:  2
subset:  3,6 round:  0
subset:  3,6 round:  1
subset:  3,6 round:  2
subset:  6,9 round:  0
subset:  6,9 round:  1
subset:  6,9 round:  2
